In [ ]:
# Import the necessary libraries and set up environment variables
# This cell initializes the required packages and defines API token and main currency
import os
import pandas as pd
import numpy as np
import requests
import json


api_token = os.getenv('EODHD_API_TOKEN')
main_currency = 'EUR'


In [ ]:
# Load portfolio data from CSV file and convert to dictionary format
# This cell reads the portfolio data and transforms it into a more accessible structure
df = pd.read_csv('portfolio_1.csv')
portfolio = {row['asset']: row.to_dict() for _, row in df.iterrows()}
df


In [ ]:
# Fetch and update fundamental data for each asset in the portfolio
# This cell retrieves asset information like name, country, type, and sector from the API
def update_fundamentals(api_token, portfolio):

    for asset, details in portfolio.items():
        try:
            # get fundamental data
            url = f'https://eodhd.com/api/fundamentals/{asset}'
            query = {'api_token': api_token, "fmt": "json"}
            response = requests.get(url, params=query)
            fundamentals = response.json()
            portfolio[asset].update({'fundamentals': fundamentals})
            portfolio[asset]['name'] = fundamentals.get('General', {}).get('Name', asset)
            portfolio[asset]['country'] = fundamentals.get('General', {}).get('CountryISO', 'Unknown')
            portfolio[asset]['type'] = fundamentals.get('General', {}).get('Type', 'Unknown')
            portfolio[asset]['currency'] = fundamentals.get('General', {}).get('CurrencyCode',
                                                                            portfolio[asset]['currency'])

            if portfolio[asset]['type'] == 'ETF':
                if pd.isna(portfolio[asset]['country_exposure']):
                    portfolio[asset]['country_exposure'] = fundamentals.get('ETF_Data', {}).get('World_Regions', 'Unknown')
                else:
                    print(f'Country exposure already set for {asset} as ', portfolio
                    [asset]['country_exposure'])
                if pd.isna(portfolio[asset]['sector']):
                    portfolio[asset]['sector'] = fundamentals.get('ETF_Data', {}).get('Sector_Weights', 'Unknown')
                else:
                    print(f'Sector exposure already set for {asset} as ', portfolio[asset]['sector'])
            elif portfolio[asset]['type'] == 'Common Stock':
                portfolio[asset]['country_exposure'] = 'Undefined'
                portfolio[asset]['sector'] = fundamentals.get('General', {}).get('Sector', 'Undefined')
            elif portfolio[asset]['type'] == 'Crypto':
                portfolio[asset]['sector'] = fundamentals.get('General', {}).get('Sector', 'Crypto')
            else:
                print(f'Cannot identify country exposure for {asset} with type {portfolio[asset]["type"]}.')

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {asset}: {str(e)}")
    return portfolio

portfolio = update_fundamentals(api_token, portfolio)

In [ ]:
# Retrieve historical price data and calculate current values for each asset
# This cell fetches price history, computes last price, change percentage, total value, and profit/loss
def get_prices(api_token, portfolio):
    dfs_to_concat = []
    for asset, details in portfolio.items():
        try:
            # get prices history and last price
            url = f'https://eodhd.com/api/eod/{asset}'
            query = {'api_token': api_token, "fmt": "json"}
            response = requests.get(url, params=query)
            if response.status_code != 200:
                print(f"Error fetching data for {asset}: {response.status_code}")
                portfolio[asset]['previous_price'] = np.nan
                portfolio[asset]['last_price'] = np.nan
                portfolio[asset]['change_perc'] = np.nan
                portfolio[asset]['total_value'] = np.nan
                portfolio[asset]['pnl'] = np.nan
            else:
                data_eod_history = response.json()
                df_prices = pd.DataFrame(data_eod_history)
                df_prices['date'] = pd.to_datetime(df_prices['date'])
                df_prices = df_prices.set_index('date')['adjusted_close']
                portfolio[asset]['last_price'] = df_prices.sort_index().iloc[-1]
                portfolio[asset]['previous_price'] = df_prices.sort_index().iloc[-2]
                portfolio[asset]['change_perc'] = round((portfolio[asset]['last_price'] - portfolio[asset]['previous_price']) / portfolio[asset]['previous_price'] * 100, 2)
                portfolio[asset]['total_value'] = portfolio[asset]['last_price'] * portfolio[asset]['quantity']
                portfolio[asset]['pnl'] = portfolio[asset]['total_value'] - (portfolio[asset]['avg_weighted_purchase_price']*portfolio[asset]['quantity'])
                df_prices = df_prices.to_frame(name=f'{asset}')
                dfs_to_concat.append(df_prices)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {asset}: {str(e)}")

    prices = pd.concat(dfs_to_concat, axis=1)
    prices = prices.ffill()

    return portfolio, prices

portfolio, prices = get_prices(api_token, portfolio)

In [54]:
# Fetch currency exchange rates for all currencies in the portfolio
# This cell identifies unique currencies, retrieves their exchange rates, and calculates conversion rates to the main currency
def get_currencies(portfolio, api_token, main_currency='EUR'):
    unique_currencies = list(
        {str(data.get('currency')).strip() for data in portfolio.values() if pd.notna(data.get('currency'))})
    if main_currency not in unique_currencies:
        unique_currencies.append(main_currency)

    currencies = {}
    for currency in unique_currencies:
        currencies[currency] = {}

        url = f'https://eodhd.com/api/eod/{currency}.FOREX?order=d'
        query = {'api_token': api_token, "fmt": "json"}
        response = requests.get(url, params=query)
        if response.status_code != 200:
            print(f"Error fetching data for USD/{currency}: {response.status_code}")
            currencies[currency]['rate_to_USD'] = np.nan
        else:
            data = response.json()
            currencies[currency]['rate_to_USD'] = data[0]['close']

    for currency in unique_currencies:
        if currency == main_currency:
            currencies[currency]['rate_to_main'] = 1
            continue
        else:
            if currency == 'USD':
                currencies[currency]['rate_to_main'] = 1 / currencies[main_currency]['rate_to_USD']
            else:
                currencies[currency]['rate_to_main'] = currencies[currency]['rate_to_USD'] / currencies[main_currency][
                    'rate_to_USD']
    return currencies

currencies = get_currencies(portfolio, api_token, main_currency)
currencies

{'USD': {'rate_to_USD': 1, 'rate_to_main': 1.1296882060551288},
 'EUR': {'rate_to_USD': 0.8852, 'rate_to_main': 1}}

In [ ]:
# Convert all asset values to the main currency using exchange rates
# This cell calculates total value and profit/loss in the portfolio's main currency
def update_value_in_main_currency(portfolio, prices, currencies):
    for asset, details in portfolio.items():
        if pd.notna(details.get('currency')):
            portfolio[asset]['total_value_in_main_currency'] = portfolio[asset]['total_value'] / currencies[details['currency']]['rate_to_main']
            portfolio[asset]['pnl_in_main_currency'] = portfolio[asset]['pnl'] / currencies[details['currency']]['rate_to_main']
        else:
            raise ValueError(f"Currency not defined for asset {asset}")

    return portfolio

portfolio = update_value_in_main_currency(portfolio, prices, currencies)

In [ ]:
# Calculate the weight of each asset in the portfolio
# This cell determines what percentage of the total portfolio value each asset represents
def update_weights(portfolio):
    total_portfolio_value = sum(asset['total_value_in_main_currency'] for asset in portfolio.values())
    for asset in portfolio.values():
        asset['weight'] = round((asset['total_value_in_main_currency'] / total_portfolio_value) * 100, 2)
    return portfolio

portfolio = update_weights(portfolio)

In [ ]:
# Save the processed portfolio data to files for use in the dashboard
# This cell exports the portfolio, prices, and currency data to JSON and CSV files
with open("portfolio.json", 'w') as f:
    json.dump(portfolio, f, indent=4)

prices.to_csv('prices.csv')

with open("currencies.json", 'w') as f:
    json.dump(currencies, f, indent=4)
